In [42]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop

import io
import numpy as np
import pandas as pd
def create_sequences(text, sequence_length, step):
    sequences = []
    next_chars = []
    for i in range(0, len(text) - sequence_length, step):
        sequences.append(text[i: i + sequence_length])
        next_chars.append(text[i + sequence_length])
    return sequences, next_chars

def extract_characters(text):
    return sorted(list(set(text)))


def get_chars_index_dicts(chars):
    return dict((c, i) for i, c in enumerate(chars)), dict((i, c) for i, c in enumerate(chars))


def read_corpus(path):
    with io.open(path, 'r', encoding='utf8') as f:
        return f.read().lower()


def vectorize(sequences, sequence_length, chars, char_to_index, next_chars):
    X = np.zeros((len(sequences), sequence_length, len(chars)), dtype=np.bool)
    y = np.zeros((len(sequences), len(chars)), dtype=np.bool)
    for i, sentence in enumerate(sequences):
        for t, char in enumerate(sentence):
            X[i, t, char_to_index[char]] = 1
        y[i, char_to_index[next_chars[i]]] = 1

    return X, y


def build_model(sequence_length, chars):
    model = Sequential()
    model.add(LSTM(128, input_shape=(sequence_length, len(chars))))
    model.add(Dense(len(chars)))
    model.add(Activation('softmax'))

    optimizer = RMSprop(lr=0.01)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model


def sample(preds, temperature=1.0):

    if temperature == 0:
        temperature = 1

    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [28]:
SEQUENCE_LENGTH = 40
SEQUENCE_STEP = 3
PATH_TO_CORPUS = "eminem_lyrics.txt"
EPOCHS = 12
DIVERSITY = 0.3

In [29]:
text = read_corpus(PATH_TO_CORPUS)
chars = extract_characters(text)

sequences, next_chars = create_sequences(text, SEQUENCE_LENGTH, SEQUENCE_STEP)
char_to_index, indices_char = get_chars_index_dicts(chars)

X, y = vectorize(sequences, SEQUENCE_LENGTH, chars, char_to_index, next_chars)

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [31]:
from __future__ import print_function
import numpy as np
import random
import sys
from keras.models import load_model

model = build_model(SEQUENCE_LENGTH, chars)
model.fit(X_train, y_train, batch_size=128, nb_epoch=EPOCHS)

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/12
470730/470730 [==============================] - 247s 524us/step - loss: 1.8745
Epoch 2/12
470730/470730 [==============================] - 245s 521us/step - loss: 1.6612
Epoch 3/12
470730/470730 [==============================] - 245s 521us/step - loss: 2.3614
Epoch 4/12
470730/470730 [==============================] - 246s 523us/step - loss: 3.6928
Epoch 5/12
470730/470730 [==============================] - 245s 521us/step - loss: 6.0779s - loss: 6
Epoch 6/12
470730/470730 [==============================] - 246s 522us/step - loss: 8.7111
Epoch 7/12
470730/470730 [==============================] - 245s 521us/step - loss: 1.6172s - loss: 
Epoch 8/12
470730/470730 [==============================] - 245s 521us/step - loss: 1.5911
Epoch 9/12
470730/470730 [==============================] - 289s 613us/step - loss: 1.5811
Epoch 10/12
470730/470730 [==============================] - 267s 568us/step - loss: 1.5800
Epoch 11/12
470730/470730 [==============================] - 267s 56

In [57]:
lyrics = pd.read_csv("lyrics.csv")
lyrics_sel = lyrics[lyrics['artist']=="eminem"].reset_index(drop=True)['lyrics'][10:20].reset_index(drop=True)
for i in range(10):
    lyrics_sel[i] = lyrics_sel.loc[i][0:40]
lyrics_sel

0    Oh oh\nOh(yea) oh(yea) oh(yea)\nOh oh\nOh\nO
1       Good evening!\nThis is your fucking capta
2     [Eminem]\nHere I Go\nLyrical\nI have no Ide
3      [Intro]\n([Jay-Z]Best rapper alive) x4\n[J
4       Yea, (oh, oh), yea yea, oh\nI feel like d
5       It fits perfect\nBetter watch out sucka n
6      Hey!\nHey Em what's up?\nSteve Berman what
7      Yeah, yeah,\nFollow me, follow me\nCome wi
8       It may not mean nothing to y'all,\nBut un
9       A lot of people ask me\nWhere the fuck I'
Name: lyrics, dtype: object

In [58]:
out_tb_1 = {'start': lyrics_sel}
out_tb_1 = pd.DataFrame(data = out_tb_1)
out_tb_1['lyrics']=""

out_tb_2 = {'start': lyrics_sel}
out_tb_2 = pd.DataFrame(data = out_tb_1)
out_tb_2['lyrics']=""

out_tb_3 = {'start': lyrics_sel}
out_tb_3 = pd.DataFrame(data = out_tb_1)
out_tb_3['lyrics']=""

out_tb_4 = {'start': lyrics_sel}
out_tb_4 = pd.DataFrame(data = out_tb_1)
out_tb_4['lyrics']=""

ind = 0
for start_s in out_tb_1['start']:
    sentence_list = []
    for diversity in [0.2, 0.4, 0.5, 1.0]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        # insert your 40-chars long string. OBS it needs to be exactly 40 chars!
        sentence = start_s
        sentence = sentence.lower()
        generated += sentence

        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)
        
        for i in range(400):
            x = np.zeros((1, SEQUENCE_LENGTH, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_to_index[char]] = 1.

            predictions = model.predict(x, verbose=0)[0]
            next_index = sample(predictions, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

        sentence_list.append(generated)
        print()
    out_tb_1['lyrics'][ind] = sentence_list[0]
    out_tb_2['lyrics'][ind] = sentence_list[1]
    out_tb_3['lyrics'][ind] = sentence_list[2]
    out_tb_4['lyrics'][ind] = sentence_list[3]
    ind = ind +1


----- diversity: 0.2
----- Generating with seed: "oh oh
oh(yea) oh(yea) oh(yea)
oh oh
oh
o"
oh oh
oh(yea) oh(yea) oh(yea)
oh oh
oh
one to be to should and the stuck and start the fucking to start
i can start the boot and the part in the samet
i don't start me in the motherfucking to be the back to start your strips
i was the street to start the fucking sentress


/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/ipykernel/__main__.py:58: RuntimeWarning: divide by zero encountered in log


i don't start your fucking bust the street the fuck it is a stoped
i got to start the fuck it is the street and the fucking to start
i got the fuck it in the street the

----- diversity: 0.4
----- Generating with seed: "oh oh
oh(yea) oh(yea) oh(yea)
oh oh
oh
o"
oh oh
oh(yea) oh(yea) oh(yea)
oh oh
oh
of better in the fuck of me
i got the game in the wait your stric, show you still with thi©-pirs of your fucki`? i don't seem down
i got the block booe it was the fucking all the chose
i'm feel scanse for the stream but the fucking too see it at me so inside of the choses
the fuck in the waind and starting
in a fuckin' happed t©!
we can't thind in the birl, that i'm don't still it wast and start an

----- diversity: 0.5
----- Generating with seed: "oh oh
oh(yea) oh(yea) oh(yea)
oh oh
oh
o"
oh oh
oh(yea) oh(yea) oh(yea)
oh oh
oh
oh, to fuck it, not for the coof the were what you haps to seen
the time i'm /-lou3in a streind it house and gre to bust mather
i'm not hear the shit it for me in th

i feel like dreat and start the street in the people)
i don't got the changed to start the back in the morna i don't be to start the blade
i got the street and the street in the fuck
i got the more is me to start and start the choses
i don't start me the fuck it is the fucking blown
i got the bark and start the back in the block and the more
i'm a start in the blade
i got a morna and shot the street and the st

----- diversity: 0.4
----- Generating with seed: "yea, (oh, oh), yea yea, oh
i feel like d"
yea, (oh, oh), yea yea, oh
i feel like dread by the worl (my booe the fuck and the caller th8 minar
and is the fuck i don't got to the bittle
i'm the glant the tead start of the whores
i got to shit a port and dread
i'm gonna still in the bratere with the bark in the choof
i gotta startulidy you the fucking to god
start you cats and start it i the warn and the bring
when i gottin' in the`-chere in the block and like it is a man
one to s

----- diversity: 0.5
----- Generating with seed: "y

re-e not her recogmin to so ever loonser
atro
kim, prorm eatts,
with my nigga goes
thosey's to done dimger my kris to gathe in byes
(i was dic, afks akin my me god' annin arourd are had and clanes
girl these spimma that her your auforh rhdmy lo

----- diversity: 0.2
----- Generating with seed: "it may not mean nothing to y'all,
but un"
it may not mean nothing to y'all,
but understand the fuck i don't got a book
i know i know the coup the fuck it is the mor4-late
-she the street the fuck it was i got a street and the stressed
i do6 it is the ground the street to be the back of the part
i still the back to shit and the crazy
i say it go the part i got the street in the block and a stread
it's a little to steatted at the back in the back to be the fuck in the chose is a

----- diversity: 0.4
----- Generating with seed: "it may not mean nothing to y'all,
but un"
it may not mean nothing to y'all,
but understime how is bre the truest the arda and the fuc5 the tater could standed to die to t

In [60]:
out_tb_1['artist'] = "Eminem"
out_tb_2['artist'] = "Eminem"
out_tb_3['artist'] = "Eminem"
out_tb_4['artist'] = "Eminem"

out_tb_1.to_csv("Eminem_div_02.csv",encoding="utf-8")
out_tb_2.to_csv("Eminem_div_04.csv",encoding="utf-8")
out_tb_3.to_csv("Eminem_div_05.csv",encoding="utf-8")
out_tb_4.to_csv("Eminem_div_10.csv",encoding="utf-8")

,start,lyrics
0,Oh oh\nOh(yea) oh(yea) oh(yea)\nOh oh\nOh\nO,oh oh\noh(yea) oh(yea) oh(yea)\noh oh\noh\non ...
1,Good evening!\nThis is your fucking capta,good evening!\nthis is your fucking capta you ...
2,[Eminem]\nHere I Go\nLyrical\nI have no Ide,"[eminem]\nhere i go\nlyrical\ni have no ides, ..."
3,[Intro]\n([Jay-Z]Best rapper alive) x4\n[J,[intro]\n([jay-z]best rapper alive) x4\n[jea)\...
4,"Yea, (oh, oh), yea yea, oh\nI feel like d","yea, (oh, oh), yea yea, oh\ni feel like dog-gh..."
5,It fits perfect\nBetter watch out sucka n,it fits perfect\nbetter watch out sucka new sm...
6,Hey!\nHey Em what's up?\nSteve Berman what,hey!\nhey em what's up?\nsteve berman what\ni ...
7,"Yeah, yeah,\nFollow me, follow me\nCome wi","yeah, yeah,\nfollow me, follow me\ncome with t..."
8,"It may not mean nothing to y'all,\nBut un","it may not mean nothing to y'all,\nbut uncawed..."
9,A lot of people ask me\nWhere the fuck I',a lot of people ask me\nwhere the fuck i'm dis...
